In [1]:
# import libraries

import pandas as pd
from scikitlearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, KFold

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Read the CSV file with a maximum of 1,227,516 rows
player_21 = pd.read_csv( '/content/drive/MyDrive/Colab Notebooks/Mid-Sem Project/players_21.csv')

player_22 = pd.read_csv( '/content/drive/MyDrive/Colab Notebooks/Mid-Sem Project/players_22.csv')

# to display all columns of the data set
pd.set_option('display.max_columns', None)
player_21 = player_21.drop(columns=['sofifa_id'])
player_22 = player_22.drop(columns=['sofifa_id'])

<ipython-input-3-9427856965d7>:4: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  player_22 = pd.read_csv( '/content/drive/MyDrive/Colab Notebooks/Mid-Sem Project/players_22.csv')


In [4]:
player_21.head()

,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,103500000.0,560000.0,33,1987-06-24,170,72,241.0,FC Barcelona,Spain Primera Division,1.0,CAM,10.0,NaN,2004-07-01,2021.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Medium/Low,Unique,Yes,138400000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Speed Drib...",85.0,92.0,91.0,95.0,38.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,32,35,24,6,11,15,14,8,NaN,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3,19+3,https://cdn.sofifa.net/players/158/023/21_120.png,https://cdn.sofifa.net/teams/241/60.png,https://cdn.sofifa.net/flags/es.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,92,63000000.0,220000.0,35,1985-02-05,187,83,45.0,Juventus,Italian Serie A,1.0,LS,7.0,NaN,2018-07-10,2022.0,38,Portugal,1354.0,LS,7.0,Right,4,5,5,High/Low,Unique,Yes,75900000.0,"#Aerial Threat, #Dribbler, #Distance Shooter, ...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",89.0,93.0,81.0,89.0,35.0,77.0,84,95,90,82,86,88,81,76,77,92,87,91,87,95,71,94,95,84,78,93,63,29,95,82,84,95,28,32,24,7,11,15,14,11,NaN,91+1,91+1,91+1,89,91,91,91,89,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3,20+3,https://cdn.sofifa.net/players/020/801/21_120.png,https://cdn.sofifa.net/teams/45/60.png,https://cdn.sofifa.net/flags/it.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
2,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,91,91,111000000.0,240000.0,31,1988-08-21,184,80,21.0,FC Bayern München,German 1. Bundesliga,1.0,ST,9.0,NaN,2014-07-01,2023.0,37,Poland,NaN,NaN,NaN,Right,4,4,4,High/Medium,Unique,Yes,132000000.0,"#Distance Shooter, #Clinical Finisher","Solid Player, Finesse Shot, Outside Foot Shot,...",78.0,91.0,78.0,85.0,43.0,82.0,71,94,85,84,89,85,79,85,70,88,77,78,77,93,82,89,84,76,86,85,81,49,94,79,88,88,35,42,19,15,6,12,8,10,NaN,89+2,89+2,89+2,85,87,87,87,85,85+3,85+3,85+3,83+3,79+3,79+3,79+3,83+3,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/21_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,NaN,https://cdn.sofifa.net/flags/pl.png
3,https://sofifa.com/player/190871/neymar-da-sil...,Neymar Jr,Neymar da Silva Santos Júnior,"LW,

In [5]:
player_22.head()

,player_url,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url,club_logo_url,club_flag_url,nation_logo_url,nation_flag_url
0,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,93,78000000.0,320000.0,34,1987-06-24,170,72,73.0,Paris Saint-Germain,French Ligue 1,1.0,RW,30.0,NaN,2021-08-10,2023.0,52,Argentina,1369.0,RW,10.0,Left,4,4,5,Medium/Low,Unique,Yes,144300000.0,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Playmaker ...",85.0,92.0,91.0,95.0,34.0,65.0,85,95,70,91,88,96,93,94,91,96,91,80,91,94,95,86,68,72,69,94,44,40,93,95,75,96,20,35,24,6,11,15,14,8,NaN,89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,64+3,64+3,64+3,66+3,61+3,50+3,50+3,50+3,61+3,19+3,https://cdn.sofifa.net/players/158/023/22_120.png,https://cdn.sofifa.net/teams/73/60.png,https://cdn.sofifa.net/flags/fr.png,https://cdn.sofifa.net/teams/1369/60.png,https://cdn.sofifa.net/flags/ar.png
1,https://sofifa.com/player/188545/robert-lewand...,R. Lewandowski,Robert Lewandowski,ST,92,92,119500000.0,270000.0,32,1988-08-21,185,81,21.0,FC Bayern München,German 1. Bundesliga,1.0,ST,9.0,NaN,2014-07-01,2023.0,37,Poland,1353.0,RS,9.0,Right,4,4,5,High/Medium,Unique,Yes,197200000.0,"#Aerial Threat, #Distance Shooter, #Clinical F...","Solid Player, Finesse Shot, Outside Foot Shot,...",78.0,92.0,79.0,86.0,44.0,82.0,71,95,90,85,89,85,79,85,70,88,77,79,77,93,82,90,85,76,86,87,81,49,95,81,90,88,35,42,19,15,6,12,8,10,NaN,90+2,90+2,90+2,85,88,88,88,85,86+3,86+3,86+3,84+3,80+3,80+3,80+3,84+3,64+3,66+3,66+3,66+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3,https://cdn.sofifa.net/players/188/545/22_120.png,https://cdn.sofifa.net/teams/21/60.png,https://cdn.sofifa.net/flags/de.png,https://cdn.sofifa.net/teams/1353/60.png,https://cdn.sofifa.net/flags/pl.png
2,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",91,91,45000000.0,270000.0,36,1985-02-05,187,83,11.0,Manchester United,English Premier League,1.0,ST,7.0,NaN,2021-08-27,2023.0,38,Portugal,1354.0,ST,7.0,Right,4,5,5,High/Low,Unique,Yes,83300000.0,"#Aerial Threat, #Dribbler, #Distance Shooter, ...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",87.0,94.0,80.0,88.0,34.0,75.0,87,95,90,80,86,88,81,84,77,88,85,88,86,94,74,94,95,77,77,93,63,29,95,76,88,95,24,32,24,7,11,15,14,11,NaN,90+1,90+1,90+1,88,89,89,89,88,86+3,86+3,86+3,86+3,78+3,78+3,78+3,86+3,63+3,59+3,59+3,59+3,63+3,60+3,53+3,53+3,53+3,60+3,20+3,https://cdn.sofifa.net/players/020/801/22_120.png,https://cdn.sofifa.net/teams/11/60.png,https://cdn.sofifa.net/flags/gb-eng.png,https://cdn.sofifa.net/teams/1354/60.png,https://cdn.sofifa.net/flags/pt.png
3,https://sofifa.com/player/19

In [6]:
player_21.describe()

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,club_contract_valid_until,nationality_id,nation_team_id,nation_jersey_number,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,18944.000000,18944.000000,1.870700e+04,18719.000000,18944.000000,18944.000000,18944.000000,18719.000000,18719.000000,18719.000000,18719.000000,18944.000000,1127.000000,1127.000000,18944.000000,18944.000000,18944.000000,1.794900e+04,16861.000000,16861.000000,16861.000000,16861.000000,16861.000000,16861.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,18944.000000,2083.000000
mean,65.677787,71.086729,2.902288e+06,9148.482825,25.225823,181.190773,75.016892,47908.905551,1.355468,20.589668,2021.983332,57.107791,32631.673469,12.029281,2.936603,2.363017,1.091850,4.296353e+06,67.668110,52.274954,57.139434,62.455430,51.316292,64.458988,49.612859,45.796083,51.874578,58.705448,42.669552,55.546400,47.187289,42.356155,52.653030,58.482791,64.285103,64.326489,63.333193,61.609428,63.918549,57.752798,64.590530,62.596020,64.742927,46.758763,55.488862,46.250000,50.255807,53.828231,48.050412,57.978674,46.470017,47.581767,45.546505,16.446052,16.236486,16.103357,16.225982,16.551309,37.203553
std,7.002278,6.109985,7.743775e+06,19893.149956,4.697354,6.825672,7.057140,53906.733776,0.739015,17.057568,1.260426,48.696159,49455.505830,6.674627,0.667132,0.766469,0.361841,1.005968e+07,10.984862,13.991421,10.273655,10.048706,16.405179,9.746706,18.153057,19.592269,17.321709,14.572383,17.661569,18.787112,18.231204,17.239561,15.205281,16.598040,14.926452,14.689422,14.625189,9.112954,14.075215,13.330631,11.888479,15.878793,12.519775,19.305319,17.203045,20.738106,19.443738,13.724072,15.671721,12.118390,20.169591,21.402461,20.953997,17.577332,16.845480,16.519399,17.017341,17.878121,10.714523
min,47.000000,47.000000,9.000000e+03,500.000000,16.000000,155.000000,50.000000,1.000000,1.000000,1.000000,2020.000000,1.000000,1318.000000,1.000000,1.000000,1.000000,1.000000,9.000000e+03,25.000000,16.000000,25.000000,25.000000,15.000000,28.000000,6.000000,3.000000,5.000000,7.000000,3.000000,5.000000,4.000000,5.000000,5.000000,5.000000,13.000000,12.000000,14.000000,24.000000,12.000000,18.000000,15.000000,12.000000,16.000000,4.000000,9.000000,3.000000,2.000000,9.000000,6.000000,12.000000,3.000000,5.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,461.000000,1.000000,9.000000,2021.000000,21.000000,1352.000000,6.000000,3.000000,2.000000,1.000000,5.250000e+05,62.000000,42.000000,50.000000,57.000000,36.000000,58.000000,38.000000,30.000000,44.000000,54.000000,30.000000,49.000000,35.000000,31.000000,43.000000,54.000000,57.000000,57.000000,55.000000,56.000000,56.000000,48.000000,58.000000,55.000000,57.000000,32.000000,44.000000

In [7]:
player_22.describe()

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,club_team_id,league_level,club_jersey_number,club_contract_valid_until,nationality_id,nation_team_id,nation_jersey_number,weak_foot,skill_moves,international_reputation,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,19239.000000,19239.000000,1.916500e+04,19178.000000,19239.000000,19239.000000,19239.000000,19178.000000,19178.000000,19178.000000,19178.000000,19239.000000,759.000000,759.000000,19239.000000,19239.000000,19239.000000,1.806300e+04,17107.000000,17107.000000,17107.000000,17107.000000,17107.000000,17107.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,19239.000000,2132.000000
mean,65.772182,71.079370,2.850452e+06,9017.989363,25.210822,181.299704,74.943032,50580.498123,1.354364,20.945250,2022.764000,58.602682,14480.848485,12.567852,2.946151,2.352461,1.094184,5.374044e+06,68.213071,52.345297,57.312562,62.561174,51.703630,64.823289,49.577421,45.894433,51.783877,58.867977,42.463849,55.660429,47.268933,42.249025,53.072249,58.472010,64.652893,64.714902,63.500078,61.450023,64.068611,57.776860,64.813504,63.084880,65.007745,46.642705,55.538957,46.613545,50.330215,53.964603,47.858724,57.929830,46.601746,48.045584,45.906700,16.406102,16.192474,16.055356,16.229274,16.491814,36.439962
std,6.880232,6.086213,7.613700e+06,19470.176724,4.748235,6.863179,7.069434,54401.868535,0.747865,17.909369,1.213203,50.298614,35328.730217,7.039116,0.671560,0.767659,0.371098,1.494837e+07,10.933155,14.051623,10.068965,9.651312,16.189746,9.791886,18.034661,19.721023,17.294183,14.490858,17.653329,18.784590,18.181085,17.178590,15.026569,16.663722,15.167399,14.965426,14.862285,9.042281,14.324789,13.192224,12.122977,16.145279,12.663518,19.411583,16.972181,20.677077,19.621601,13.650481,15.768583,12.159326,20.200807,21.232718,20.755683,17.574028,16.839528,16.564554,17.059779,17.884833,10.751563
min,47.000000,49.000000,9.000000e+03,500.000000,16.000000,155.000000,49.000000,1.000000,1.000000,1.000000,2021.000000,1.000000,1318.000000,1.000000,1.000000,1.000000,1.000000,1.600000e+04,28.000000,18.000000,25.000000,27.000000,14.000000,29.000000,6.000000,2.000000,5.000000,7.000000,3.000000,4.000000,6.000000,4.000000,9.000000,8.000000,14.000000,15.000000,18.000000,25.000000,15.000000,20.000000,22.000000,12.000000,19.000000,4.000000,10.000000,3.000000,2.000000,10.000000,7.000000,12.000000,4.000000,5.000000,5.000000,2.000000,2.000000,2.000000,2.000000,2.000000,15.000000
25%,61.000000,67.000000,4.750000e+05,1000.000000,21.000000,176.000000,70.000000,479.000000,1.000000,9.000000,2022.000000,21.000000,1338.000000,7.000000,3.000000,2.000000,1.000000,8.060000e+05,62.000000,42.000000,51.000000,57.000000,37.000000,59.000000,38.000000,30.000000,44.000000,54.000000,30.000000,50.000000,35.000000,31.000000,44.000000,55.000000,57.000000,58.000000,55.000000,56.000000,56.000000,48.000000,57.000000,56.000000,57.000000,32.000000,44.000000,

In [9]:
## Dropping irrelevant columns: Categorical features
cat_21 = player_21.select_dtypes(include=['object']).columns
cat_22 = player_22.select_dtypes(include=['object']).columns

#Skipping URLS
t = ['url' not in c for c in cat_21]
t2 = ['url' not in c for c in cat_22]

cat_21 = cat_21[t]
cat_21 = cat_22[t2]


FEATURE EXTRACTIONS FOR TEXT

In [10]:
player_21[cat_21]

,short_name,long_name,player_positions,dob,club_name,league_name,club_position,club_loaned_from,club_joined,nationality_name,nation_position,preferred_foot,work_rate,body_type,real_face,player_tags,player_traits,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,L. Messi,Lionel Andrés Messi Cuccittini,"RW, ST, CF",1987-06-24,FC Barcelona,Spain Primera Division,CAM,NaN,2004-07-01,Argentina,RW,Left,Medium/Low,Unique,Yes,"#Dribbler, #Distance Shooter, #FK Specialist, ...","Finesse Shot, Long Shot Taker (AI), Speed Drib...",89+3,89+3,89+3,92,93,93,93,92,93,93,93,91+2,87+3,87+3,87+3,91+2,66+3,65+3,65+3,65+3,66+3,62+3,52+3,52+3,52+3,62+3,19+3
1,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",1985-02-05,Juventus,Italian Serie A,LS,NaN,2018-07-10,Portugal,LS,Right,High/Low,Unique,Yes,"#Aerial Threat, #Dribbler, #Distance Shooter, ...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",91+1,91+1,91+1,89,91,91,91,89,88+3,88+3,88+3,88+3,81+3,81+3,81+3,88+3,65+3,61+3,61+3,61+3,65+3,61+3,54+3,54+3,54+3,61+3,20+3
2,R. Lewandowski,Robert Lewandowski,ST,1988-08-21,FC Bayern München,German 1. Bundesliga,ST,NaN,2014-07-01,Poland,NaN,Right,High/Medium,Unique,Yes,"#Distance Shooter, #Clinical Finisher","Solid Player, Finesse Shot, Outside Foot Shot,...",89+2,89+2,89+2,85,87,87,87,85,85+3,85+3,85+3,83+3,79+3,79+3,79+3,83+3,64+3,65+3,65+3,65+3,64+3,61+3,60+3,60+3,60+3,61+3,19+3
3,Neymar Jr,Neymar da Silva Santos Júnior,"LW, CAM",1992-02-05,Paris Saint-Germain,French Ligue 1,LW,NaN,2017-08-03,Brazil,NaN,Right,High/Medium,Unique,Yes,"#Speedster, #Dribbler, #Playmaker, #FK Special...","Injury Prone, Flair, Speed Dribbler (AI), Outs...",84+3,84+3,84+3,90,89,89,89,90,90+1,90+1,90+1,90+1,83+3,83+3,83+3,90+1,67+3,62+3,62+3,62+3,67+3,62+3,49+3,49+3,49+3,62+3,20+3
4,K. De Bruyne,Kevin De Bruyne,"CAM, CM",1991-06-28,Manchester City,English Premier League,RCM,NaN,2015-08-30,Belgium,RCM,Right,High/High,Unique,Yes,"#Dribbler, #Playmaker, #Engine, #Distance Shoo...","Injury Prone, Leadership, Early Crosser, Long ...",83+3,83+3,83+3,88,88,88,88,88,89+2,89+2,89+2,89+2,89+2,89+2,89+2,89+2,79+3,80+3,80+3,80+3,79+3,75+3,69+3,69+3,69+3,75+3,21+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18939,Zhang Mengxuan,张梦炫,CB,1999-04-26,Chongqing Liangjiang Athletic,Chinese Super League,SUB,NaN,2020-08-01,China PR,NaN,Right,Low/Low,Normal (170-185),No,NaN,NaN,32+2,32+2,32+2,30,30,30,30,30,29+2,29+2,29+2,31+2,32+2,32+2,32+2,31+2,42+2,41+2,41+2,41+2,42+2,45+2,47+2,47+2,47+2,45+2,15+2
18940,Huang Wenzhou,黄文卓,CM,1999-01-07,Shanghai Port FC,Chinese Super League,RES,NaN,2020-08-01,China PR,NaN,Right,Low/Low,Lean (170-185),No,NaN,NaN,41+2,41+2,41+2,45,44,44,44,45,46+2,46+2,46+2,46+2,47+2,47+2,47+2,46+2,47+2,48+2,48+2,48+2,47+2,47+2,46+2,46+2,46+2,47+2,15+2
18941,Song Yue,宋岳,CM,1991-11-20,Tianjin Jinmen Tiger FC,Chinese Super League,RES,NaN,2020-08-01,China PR,NaN,Right,Low/Low,Lean (185+),No,NaN,NaN,46+1,46+1,46+1,47,46,46,46,47,47,47,47,49-2,47,47,47,49-2,47,47,47,47,47,47,46+1,46+1,46+1,47,11+2
18942,V. Da Silva,Ivanilson Loforte Tique Da Silva,ST,2003-03-30,Oldham Athletic,English League Two,SUB,NaN,2020-08-01,England,NaN,Right,Medium/Medium,Lean (170-185),No,NaN,NaN,47+2,47+2,47+2,49,49,49,49,49,49+2,49+2,49+2,48+2,43+2,43+2,43+2,48+2,35+2,32+2,32+2,32+2,35+2,33+2,26+2,26+2,26+2,33+2,14+2


In [13]:
### Impute missing values in

## players_21:
# Extract numerical features
num_21 = player_21.select_dtypes(include=['int64', 'float64']).columns
# Impute missing values in players_21
num_imputer_21 = SimpleImputer(strategy='mean')

player_21[num_21] = num_imputer_21.fit_transform(player_21[num_21])

## players_22:
# Extract numerical features
num_22 = player_22.select_dtypes(include=['int64', 'float64']).columns
# Impute missing values in players_22
num_imputer_22 = SimpleImputer(strategy='mean')
player_22[num_22] = num_imputer_22.fit_transform(player_22[num_22])


#Filling in NaNs for text values (imputing with forward fill)
player_21[cat_21].fillna(method='ffill', inplace=True)
player_22[cat_22].fillna(method='ffill', inplace=True)


#coverting texts into numerical values with encoder
enc = LabelEncoder()
for c in player_21[cat_21]:
  player_21[c] = enc.fit_transform(player_21[c])


enc2 = LabelEncoder()
for c2 in player_22[cat_22]:
  player_22[c2] = enc.fit_transform(player_22[c2])

# Merging textual and numerical data for training
merge_num_text_21 = pd.concat([player_21[num_21], player_21[cat_21]], axis=0)
merge_num_text_22 = pd.concat([player_22[num_22], player_22[cat_22]], axis=0)


<ipython-input-13-93afeb7830dc>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_21[cat_21].fillna(method='ffill', inplace=True)
<ipython-input-13-93afeb7830dc>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_22[cat_22].fillna(method='ffill', inplace=True)


In [ ]:
# nanValues = merge_num_text_21.isna()
# merge_num_text_21= merge_num_text_21[nanValues]

Using correlation to find the dependent variables that are strongly contributing to the players overall
INDEPENDENT VARIABLE: Overall
DEPENDENT: Remaining variables


In [14]:
corr_matrix = merge_num_text_21.corr()
corr_matrix_22 = player_22.corr()
corr_matrix



,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,club_team_id,club_name,league_name,league_level,club_position,club_jersey_number,club_loaned_from,club_joined,club_contract_valid_until,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
short_name,1.000000,0.867243,-0.004629,0.013936,-0.005557,0.016025,0.014151,0.019558,-0.019905,0.008148,0.001712,-0.002791,0.040814,-0.018636,-0.007948,-0.003334,-0.021485,-0.007098,-0.018309,-0.014131,0.047309,-0.009906,-0.000927,0.003299,-0.014230,0.023944,0.017082,-0.004289,0.011469,0.012342,-0.010894,0.011304,0.019548,-0.004656,-0.004870,-0.009417,0.001863,0.011138,-0.001082,0.018831,0.009710,0.000446,-0.002422,-0.001430,0.006084,0.001563,-0.007194,-0.000906,0.000367,0.012599,-0.000124,-0.008392,-0.007633,-0.004793,0.015378,-0.001503,0.000286,0.002765,0.005625,0.009314,-0.000308,0.005282,0.014686,-0.002350,0.006256,-0.010313,0.010683,0.013509,0.011330,0.008690,0.007398,0.005874,0.005584,0.006641,0.008012,9.879291e-03,0.000654,0.000654,0.000654,0.003038,0.002758,0.002758,0.002758,0.003038,0.004758,0.004758,0.004758,0.004429,0.011264,0.011264,0.011264,0.004429,0.013068,0.014990,0.014990,0.014990,0.013068,0.013097,0.012946,0.012946,0.012946,0.013097,0.009784
long_name,0.867243,1.000000,-0.007649,-0.028753,-0.071836,-0.008746,-0.011800,0.040696,-0.041021,-0.001196,-0.013169,0.050595,0.062358,-0.049827,-0.038565,-0.004652,-0.021113,-0.014974,-0.043846,-0.026600,0.197719,-0.036786,-0.006256,0.014875,-0.011608,0.037115,0.016407,-0.026507,-0.001218,0.013392,-0.029753,-0.006680,-0.011583,0.007825,0.011977,-0.014131,-0.021619,-0.010502,-0.029835,0.012924,0.003132,-0.013341,-0.020024,-0.016726,-0.014889,-0.019326,-0.027756,-0.023614,-0.011862,-0.001332,-0.023322,-0.017251,-0.014923,-0.011953,-0.014568,-0.004545,-0.032340,0.003238,-0.001551,0.003352,-0.017442,-0.007454,0.010502,-0.012359,-0.005790,-0.035958,-0.019303,0.007807,0.004963,0.002354,0.007646,0.006045,0.006564,0.006437,0.008573,-1.207305e-03,-0.025225,-0.025225,-0.025225,-0.022142,-0.023296,-0.023296,-0.023296,-0.022142,-0.019905,-0.019905,-0.019905,-0.019744,-0.011999,-0.011999,-0.011999,-0.019744,-0.005789,-0.003882,-0.003882,-0.003882,-0.005789,-0.004648,-0.003591,-0.003591,-0.003591,-0.004648,0.003952
player_positions,-0.004629,-0.007649,1.000000,0.009893,0.002880,0.012582,0.017083,-0.027932,0.026067,-0.140687,-0.093325,0.000217,0.006419,0.002909,0.009636,0.222786,0.054953,-0.027993,0.068073,-0.002282,0.014889,0.015210,-0.003481,0.043030,0.018403,0.057737,0.113048,0.210696,0.000355,-0.175446,-0.025752,-0.000895,0.014637,-0.048512,0.007249,0.385652,0.439384,-0.012903,0.288949,-0.587491,-0.157245,0.154170,0.440536,0.057838,-0.021225,0.353171,0.257967,0.192461,0.078505,-0.184813,0.156179,0.313503,0.322806,0.256688,0.004429,0.166951,0.257343,-0.000753,0.032674,-0.081828,0.278431,-0.178995,-0.450090,0.381179,0.126477,0.330521,0.031419,-0.417705,-0.460153,-0.465866,-0.073760,-0.072961,-0.072708,-0.072611,-0

Identifying the key variables that strongly contribute to the players overall according to the correlation analysis on the dataset.

In [15]:
corr_matrix_22['overall'].sort_values(ascending=False)
corr_matrix['overall'].sort_values(ascending=False)


overall                1.000000
movement_reactions     0.867234
mentality_composure    0.705252
passing                0.662090
lcm                    0.647694
                         ...   
player_tags           -0.229177
nation_position       -0.232328
work_rate             -0.233214
player_traits         -0.386814
dob                   -0.480212
Name: overall, Length: 103, dtype: float64

Identifying the key features that characterize the dependent variables. Dropping all other variables except the below variables obtained from the correlation analysis.

In [16]:
#Threshold for correlation relevance is set to 50%
corr_matrix['overall'] > 0.5

short_name          False
long_name           False
player_positions    False
overall              True
potential            True
                    ...  
lcb                 False
cb                  False
rcb                 False
rb                   True
gk                  False
Name: overall, Length: 103, dtype: bool

In [17]:
# List of feature names with highest correlation to 'overall'
selected_features = ['movement_reactions','passing','mentality_composure','dribbling','potential','release_clause_eur','wage_eur',
    'value_eur','power_shot_power','physic','mentality_vision','attacking_short_passing'
]
# Select these features from the dataset
selected_data = player_21[['overall'] + selected_features]

Presenting the Dependent Variables Alongside Their Various Metrics

Ensuring that all missing values have been effectively substituted with their respective means.

Dividing the dataset into two distinct components: the Y-variable, representing the target or dependent variable, and the X-variables, which encompass the independent variables or features.

In [18]:
from sklearn.model_selection import train_test_split
# Specify the features (X) and the target variable (y)
y = selected_data['overall']  # Target variable
X = selected_data.drop(columns=['overall'])  # Features


Before standardizing the dependent variables

In [34]:
X

,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power,physic,mentality_vision,attacking_short_passing
0,3.554438,3.493625,3.137573,3.433013,3.586563,13.695696,27.857178,13.073165,2.119026,0.058838,3.000047,2.216206
1,3.664174,2.461857,3.055051,2.800094,3.422893,7.312715,10.662997,7.809992,2.719163,1.363898,2.052781,1.598583
2,3.444701,2.152326,2.477402,2.378148,3.259222,13.042079,11.674420,14.047827,2.344077,1.907673,1.834181,1.735833
3,3.225227,2.977741,2.890009,3.327526,3.259222,16.565484,13.191553,16.776879,1.668922,-0.593692,2.635714,1.941707
4,3.225227,3.699979,2.724966,2.694607,3.259222,16.003782,18.248665,16.387015,2.494112,1.472653,2.927180,2.422080
...,...,...,...,...,...,...,...,...,...,...,...,...
18939,-1.493455,-3.212867,-1.483625,-3.740069,-3.123941,-0.432955,-0.412078,-0.368070,-1.706851,-1.789997,-2.100615,-1.901280
18940,-1.273981,-0.839801,-1.896232,-1.630339,-2.960270,-0.431423,-0.412078,-0.368070,-1.481799,-1.789997,-0.060350,-0.254285
18941,-1.932402,-0.839801,-1.896232,-1.735826,-3.942295,-0.433977,-0.361507,-0.371319,-0.656610,-0.593692,-0.716150,-0.185661
18942,-0.944771,-1.768392,-1.071018,-0.997420,-0.668878,-0.421926,-0.437363,-0.360272,-0.581593,-2.660037,-0.351817,-0.940533


Standardizing the dependent variables to ensure uniform measurement units across all of them.

In [20]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
scaled = sc.fit_transform(X)
X = pd.DataFrame(scaled, columns=X.columns)
X

,movement_reactions,passing,mentality_composure,dribbling,potential,release_clause_eur,wage_eur,value_eur,power_shot_power,physic,mentality_vision,attacking_short_passing
0,3.554438,3.493625,3.137573,3.433013,3.586563,13.695696,27.857178,13.073165,2.119026,0.058838,3.000047,2.216206
1,3.664174,2.461857,3.055051,2.800094,3.422893,7.312715,10.662997,7.809992,2.719163,1.363898,2.052781,1.598583
2,3.444701,2.152326,2.477402,2.378148,3.259222,13.042079,11.674420,14.047827,2.344077,1.907673,1.834181,1.735833
3,3.225227,2.977741,2.890009,3.327526,3.259222,16.565484,13.191553,16.776879,1.668922,-0.593692,2.635714,1.941707
4,3.225227,3.699979,2.724966,2.694607,3.259222,16.003782,18.248665,16.387015,2.494112,1.472653,2.927180,2.422080
...,...,...,...,...,...,...,...,...,...,...,...,...
18939,-1.493455,-3.212867,-1.483625,-3.740069,-3.123941,-0.432955,-0.412078,-0.368070,-1.706851,-1.789997,-2.100615,-1.901280
18940,-1.273981,-0.839801,-1.896232,-1.630339,-2.960270,-0.431423,-0.412078,-0.368070,-1.481799,-1.789997,-0.060350,-0.254285
18941,-1.932402,-0.839801,-1.896232,-1.735826,-3.942295,-0.433977,-0.361507,-0.371319,-0.656610,-0.593692,-0.716150,-0.185661
18942,-0.944771,-1.768392,-1.071018,-0.997420,-0.668878,-0.421926,-0.437363,-0.360272,-0.581593,-2.660037,-0.351817,-0.940533


Create the models, initialize and train the models to get ready for prediction
Using Ensemble Learning

In [21]:
# Split the data into training and testing sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
}

regressorGB = GradientBoostingRegressor()
grid_searchGB = GridSearchCV(estimator=regressorGB, param_grid=param_grid, cv=KFold(n_splits=5), scoring='neg_mean_squared_error')
grid_searchGB.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [25]:
grid_searchGB.best_score_

-1.151141867994658

In [26]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

regressorRF = RandomForestRegressor()
grid_searchRF = GridSearchCV(estimator=regressorRF, param_grid=param_grid, cv=KFold(n_splits=5), scoring='neg_mean_squared_error')
grid_searchRF.fit(X_train, y_train)

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=RandomForestRegressor(),
             param_grid={'max_depth': [None, 10, 20],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [27]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
}

regressorXGB = xgb.XGBRegressor()
grid_searchXGB = GridSearchCV(estimator=regressorXGB, param_grid=param_grid, cv=KFold(n_splits=5), scoring='neg_mean_squared_error')
grid_searchXGB.fit(X_train, y_train)


GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    i...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

In [28]:
best_model = grid_searchGB
best_params = grid_searchGB.get_params

if grid_searchGB.best_score_ > grid_searchRF.best_score_ and grid_searchGB.best_score_ > grid_searchXGB.best_score_:
    best_model = grid_searchGB
    best_params = grid_searchGB.best_params_
elif grid_searchRF.best_score_ > grid_searchGB.best_score_ and grid_searchRF.best_score_ > grid_searchXGB.best_score_:
    best_model = grid_searchRF
    best_params = grid_searchRF.best_params_
else:
    best_model = grid_searchXGB
    best_params = grid_searchXGB.best_params_


In [29]:
best_model

GridSearchCV(cv=KFold(n_splits=5, random_state=None, shuffle=False),
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    i...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 4, 5],
                         'n_estimators': [100, 200, 300]},
             scoring='neg_mean_squared_error')

SAVING THE MODEL AS A PICKLE FILE

In [30]:
import pickle

# Save the model to a file
with open('football_prediction_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)


TESTING MODEL WITH VALIDATION DATA (PLAYER_21)

In [ ]:
best_model.predict(X_test)

In [ ]:
corr_matrix_22['overall'] > 0.5

TESTING MODEL WITH TEST DATA (PLAYER_22)

In [ ]:
# List of feature names with highest correlation to 'overall'
selected_features_22 = ['movement_reactions','passing','mentality_composure','dribbling','potential','release_clause_eur','wage_eur',
    'value_eur','power_shot_power','physic','mentality_vision','attacking_short_passing'
]
# Select these features from the dataset
selected_data = player_22[['overall'] + selected_features]

# Specify the features (X) and the target variable (y)
y_22 = selected_data_22['overall']  # Target variable
X_22 = selected_data_22.drop(columns=['overall'])  # Features


sc = StandardScaler()
scaled = sc.fit_transform(X)
X_22 = pd.DataFrame(scaled, columns=X.columns)
X_22

In [ ]:
best_model.predict(X_22)